In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import pandas as pd
from PIL import Image
import os

In [3]:
train_path = '../train-set/'

In [4]:
chars = ['a','nu','ti','ke','yo','yorozu']

# Import Data

In [5]:
train_data = {}
filenames = {}
for c in chars:
    files = os.listdir(train_path + c + '/')
    train_data_ = []
    for f in files:
        img = np.asarray(Image.open(train_path + c + '/' + f).convert('L')).reshape((64,64,1))
        train_data_.append(img)
    train_data_ = np.asarray(train_data_) / 255
    train_data[c] = train_data_
    filenames[c] = files

# Autoencoder Class

In [6]:
class Autoencoder:
    def __init__(self, pretrained_path = None):
        if pretrained_path == None:
            self.shape = (64,64,1)

            encoder_input = layers.Input(shape=self.shape)
            x = layers.Conv2D(64,(3,3),padding = 'same', activation = 'relu')(encoder_input)
            x = layers.BatchNormalization()(x)
            x = layers.MaxPooling2D(pool_size=(2,2))(x)
            x = layers.Conv2D(32,(3,3),padding = 'same', activation = 'relu')(x)
            x = layers.BatchNormalization()(x)
            x = layers.MaxPooling2D(pool_size=(2,2))(x)
            x = layers.Conv2D(16,(3,3),padding = 'same', activation = 'relu')(x)
            x = layers.BatchNormalization()(x)
            x = layers.MaxPooling2D(pool_size=(2,2))(x)
            x = layers.Conv2D(16,(3,3),padding = 'same', activation = 'relu')(x)
            x = layers.BatchNormalization()(x)
            x = latent_layer = layers.MaxPooling2D(pool_size=(2,2))(x)

            decoder_input = layers.Input(shape = (4,4,16))
            y = layers.Conv2DTranspose(16,(3,3),padding = 'same', activation = 'relu')(decoder_input)
            y = layers.BatchNormalization()(y)
            y = layers.UpSampling2D((2,2))(y)
            y = layers.Conv2DTranspose(32,(3,3),padding = 'same', activation = 'relu')(y)
            y = layers.BatchNormalization()(y)
            y = layers.UpSampling2D((2,2))(y)
            y = layers.Conv2DTranspose(64,(3,3),padding = 'same', activation = 'relu')(y)
            y = layers.BatchNormalization()(y)
            y = layers.UpSampling2D((2,2))(y)
            y = layers.Conv2DTranspose(64,(3,3),padding = 'same', activation = 'relu')(y)
            y = layers.BatchNormalization()(y)
            y = layers.UpSampling2D((2,2))(y)
            output_layer = layers.Conv2DTranspose(1,(3,3),padding = 'same', activation = 'sigmoid')(y)

            self.encoder = models.Model(inputs = encoder_input, outputs = latent_layer)
            self.decoder = models.Model(inputs = decoder_input, outputs = output_layer)
            self.model = models.Sequential([self.encoder,self.decoder])

            self.model.layers[0]._name = 'encoder'
            self.model.layers[1]._name = 'decoder'

            self.model.compile(optimizer = optimizers.Adam(learning_rate = 0.1), loss='binary_crossentropy')
        else:
            self.encoder = models.load_model(pretrained_path + 'encoder.h5')
            self.decoder = models.load_model(pretrained_path + 'decoder.h5')
            self.model = models.load_model(pretrained_path + 'autoencoder.h5')
        
    def summary(self):
        self.model.summary()
            
    def encode(self, input):
        return encoder(input)
        
    def decode(self, latent_vector):
        return decoder(latent_vector)
    
    def evaluate(self, input):
        return self.model.evaluate(input,input)
    
    def reconstruct(self, input):
        return self.decoder(self.encoder(input))
        
    def fit(self, x, epochs):
        self.model.fit(x,x,epochs = epochs, batch_size = 128, shuffle = True, validation_split = 0.1)
        self.history = self.model.history
        
    def save(self, path):
        if not os.path.isdir(path):
            os.mkdir(path)
        
        self.encoder.save(path + 'encoder.h5')
        self.decoder.save(path + 'decoder.h5')
        self.model.save(path + 'autoencoder.h5')

# Train Each Model Seperately

In [7]:
chars = ['yo','yorozu']
autoencoder = {}
for c in chars:
    autoencoder[c] = Autoencoder()

In [8]:
for c in chars:
    if c == 'yo' or c == 'yorozu': epochs = 800
    else: epochs = 500
    autoencoder[c].fit(train_data[c], epochs)

Epoch 1/800
1/1 [==============================] - 14s 14s/step - loss: 0.9086 - val_loss: 5019.1294
Epoch 2/800
1/1 [==============================] - 0s 77ms/step - loss: 6.7733 - val_loss: 85112.1094
Epoch 3/800
1/1 [==============================] - 0s 77ms/step - loss: 6.2013 - val_loss: 1561091.0000
Epoch 4/800
1/1 [==============================] - 0s 83ms/step - loss: 5.8718 - val_loss: 849159.1875
Epoch 5/800
1/1 [==============================] - 0s 74ms/step - loss: 1.4247 - val_loss: 260736.9531
Epoch 6/800
1/1 [==============================] - 0s 74ms/step - loss: 0.4578 - val_loss: 47993.4492
Epoch 7/800
1/1 [==============================] - 0s 87ms/step - loss: 0.4331 - val_loss: 16086.1807
Epoch 8/800
1/1 [==============================] - 0s 69ms/step - loss: 0.4244 - val_loss: 17923.9551
Epoch 9/800
1/1 [==============================] - 0s 69ms/step - loss: 0.3717 - val_loss: 17141.4492
Epoch 10/800
1/1 [==============================] - 0s 70ms/step - loss: 0.3291

Epoch 83/800
1/1 [==============================] - 0s 70ms/step - loss: 0.1847 - val_loss: 0.5152
Epoch 84/800
1/1 [==============================] - 0s 65ms/step - loss: 0.1807 - val_loss: 0.3736
Epoch 85/800
1/1 [==============================] - 0s 68ms/step - loss: 0.1829 - val_loss: 0.5687
Epoch 86/800
1/1 [==============================] - 0s 69ms/step - loss: 0.1804 - val_loss: 0.3554
Epoch 87/800
1/1 [==============================] - 0s 67ms/step - loss: 0.1840 - val_loss: 0.6750
Epoch 88/800
1/1 [==============================] - 0s 65ms/step - loss: 0.1812 - val_loss: 0.3424
Epoch 89/800
1/1 [==============================] - 0s 64ms/step - loss: 0.1860 - val_loss: 0.7164
Epoch 90/800
1/1 [==============================] - 0s 66ms/step - loss: 0.1813 - val_loss: 0.3360
Epoch 91/800
1/1 [==============================] - 0s 65ms/step - loss: 0.1849 - val_loss: 0.6771
Epoch 92/800
1/1 [==============================] - 0s 74ms/step - loss: 0.1801 - val_loss: 0.3345
Epoch 93/8

1/1 [==============================] - 0s 66ms/step - loss: 0.1644 - val_loss: 0.2007
Epoch 166/800
1/1 [==============================] - 0s 68ms/step - loss: 0.1625 - val_loss: 0.2307
Epoch 167/800
1/1 [==============================] - 0s 67ms/step - loss: 0.1637 - val_loss: 0.1987
Epoch 168/800
1/1 [==============================] - 0s 69ms/step - loss: 0.1617 - val_loss: 0.2293
Epoch 169/800
1/1 [==============================] - 0s 64ms/step - loss: 0.1627 - val_loss: 0.1965
Epoch 170/800
1/1 [==============================] - 0s 63ms/step - loss: 0.1608 - val_loss: 0.2259
Epoch 171/800
1/1 [==============================] - 0s 72ms/step - loss: 0.1615 - val_loss: 0.1942
Epoch 172/800
1/1 [==============================] - 0s 64ms/step - loss: 0.1598 - val_loss: 0.2231
Epoch 173/800
1/1 [==============================] - 0s 57ms/step - loss: 0.1602 - val_loss: 0.1929
Epoch 174/800
1/1 [==============================] - 0s 78ms/step - loss: 0.1588 - val_loss: 0.2206
Epoch 175/800


1/1 [==============================] - 0s 69ms/step - loss: 0.1462 - val_loss: 0.2282
Epoch 248/800
1/1 [==============================] - 0s 56ms/step - loss: 0.1480 - val_loss: 0.1796
Epoch 249/800
1/1 [==============================] - 0s 65ms/step - loss: 0.1436 - val_loss: 0.2271
Epoch 250/800
1/1 [==============================] - 0s 63ms/step - loss: 0.1424 - val_loss: 0.2042
Epoch 251/800
1/1 [==============================] - 0s 64ms/step - loss: 0.1398 - val_loss: 0.2089
Epoch 252/800
1/1 [==============================] - 0s 54ms/step - loss: 0.1384 - val_loss: 0.1926
Epoch 253/800
1/1 [==============================] - 0s 52ms/step - loss: 0.1381 - val_loss: 0.1911
Epoch 254/800
1/1 [==============================] - 0s 60ms/step - loss: 0.1378 - val_loss: 0.2166
Epoch 255/800
1/1 [==============================] - 0s 60ms/step - loss: 0.1384 - val_loss: 0.1979
Epoch 256/800
1/1 [==============================] - ETA: 0s - loss: 0.139 - 0s 60ms/step - loss: 0.1391 - val_los

1/1 [==============================] - 0s 109ms/step - loss: 0.1366 - val_loss: 0.2303
Epoch 329/800
1/1 [==============================] - 0s 219ms/step - loss: 0.1357 - val_loss: 0.2579
Epoch 330/800
1/1 [==============================] - 0s 142ms/step - loss: 0.1399 - val_loss: 0.1988
Epoch 331/800
1/1 [==============================] - 0s 92ms/step - loss: 0.1318 - val_loss: 0.2869
Epoch 332/800
1/1 [==============================] - 0s 86ms/step - loss: 0.1312 - val_loss: 0.2781
Epoch 333/800
1/1 [==============================] - 0s 122ms/step - loss: 0.1289 - val_loss: 0.2021
Epoch 334/800
1/1 [==============================] - 0s 167ms/step - loss: 0.1309 - val_loss: 0.2408
Epoch 335/800
1/1 [==============================] - 0s 135ms/step - loss: 0.1346 - val_loss: 0.2233
Epoch 336/800
1/1 [==============================] - 0s 103ms/step - loss: 0.1339 - val_loss: 0.2790
Epoch 337/800
1/1 [==============================] - 0s 111ms/step - loss: 0.1370 - val_loss: 0.2114
Epoch 

...

1/1 [==============================] - 0s 70ms/step - loss: 0.0826 - val_loss: 0.3744
Epoch 424/800
1/1 [==============================] - 0s 100ms/step - loss: 0.0810 - val_loss: 0.3800
Epoch 425/800
1/1 [==============================] - 0s 151ms/step - loss: 0.0792 - val_loss: 0.3787
Epoch 426/800
1/1 [==============================] - 0s 100ms/step - loss: 0.0789 - val_loss: 0.3671
Epoch 427/800
1/1 [==============================] - 0s 100ms/step - loss: 0.0797 - val_loss: 0.3757
Epoch 428/800
1/1 [==============================] - 0s 201ms/step - loss: 0.0804 - val_loss: 0.3479
Epoch 429/800
1/1 [==============================] - 0s 159ms/step - loss: 0.0821 - val_loss: 0.3678
Epoch 430/800
1/1 [==============================] - 0s 120ms/step - loss: 0.0825 - val_loss: 0.3606
Epoch 431/800
1/1 [==============================] - 0s 139ms/step - loss: 0.0846 - val_loss: 0.3402
Epoch 432/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0839 - val_loss: 0.3665
Epoch 

1/1 [==============================] - 0s 83ms/step - loss: 0.0738 - val_loss: 0.2642
Epoch 505/800
1/1 [==============================] - 0s 120ms/step - loss: 0.0739 - val_loss: 0.2635
Epoch 506/800
1/1 [==============================] - 0s 150ms/step - loss: 0.0725 - val_loss: 0.2449
Epoch 507/800
1/1 [==============================] - 0s 100ms/step - loss: 0.0718 - val_loss: 0.2613
Epoch 508/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0724 - val_loss: 0.2368
Epoch 509/800
1/1 [==============================] - 0s 121ms/step - loss: 0.0725 - val_loss: 0.2630
Epoch 510/800
1/1 [==============================] - 0s 100ms/step - loss: 0.0708 - val_loss: 0.2680
Epoch 511/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0715 - val_loss: 0.2280
Epoch 512/800
1/1 [==============================] - 0s 100ms/step - loss: 0.0724 - val_loss: 0.2688
Epoch 513/800
1/1 [==============================] - 0s 80ms/step - loss: 0.0721 - val_loss: 0.2385
Epoch 51

1/1 [==============================] - 0s 201ms/step - loss: 0.0674 - val_loss: 0.1537
Epoch 586/800
1/1 [==============================] - 0s 150ms/step - loss: 0.0671 - val_loss: 0.1839
Epoch 587/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0665 - val_loss: 0.1561
Epoch 588/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0667 - val_loss: 0.1902
Epoch 589/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0665 - val_loss: 0.1475
Epoch 590/800
1/1 [==============================] - 0s 110ms/step - loss: 0.0659 - val_loss: 0.1778
Epoch 591/800
1/1 [==============================] - 0s 91ms/step - loss: 0.0647 - val_loss: 0.1726
Epoch 592/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0648 - val_loss: 0.1728
Epoch 593/800
1/1 [==============================] - 0s 101ms/step - loss: 0.0656 - val_loss: 0.1720
Epoch 594/800
1/1 [==============================] - 0s 191ms/step - loss: 0.0672 - val_loss: 0.1830
Epoch 595

1/1 [==============================] - 0s 141ms/step - loss: 0.0624 - val_loss: 0.1225
Epoch 667/800
1/1 [==============================] - 0s 140ms/step - loss: 0.0616 - val_loss: 0.1208
Epoch 668/800
1/1 [==============================] - 0s 141ms/step - loss: 0.0619 - val_loss: 0.1063
Epoch 669/800
1/1 [==============================] - 0s 81ms/step - loss: 0.0623 - val_loss: 0.1220
Epoch 670/800
1/1 [==============================] - 0s 70ms/step - loss: 0.0622 - val_loss: 0.1121
Epoch 671/800
1/1 [==============================] - 0s 70ms/step - loss: 0.0611 - val_loss: 0.1182
Epoch 672/800
1/1 [==============================] - 0s 80ms/step - loss: 0.0616 - val_loss: 0.1228
Epoch 673/800
1/1 [==============================] - 0s 80ms/step - loss: 0.0616 - val_loss: 0.1163
Epoch 674/800
1/1 [==============================] - 0s 81ms/step - loss: 0.0610 - val_loss: 0.1159
Epoch 675/800
1/1 [==============================] - 0s 90ms/step - loss: 0.0612 - val_loss: 0.1239
Epoch 676/8

1/1 [==============================] - 0s 90ms/step - loss: 0.0637 - val_loss: 0.1544
Epoch 748/800
1/1 [==============================] - 0s 80ms/step - loss: 0.0680 - val_loss: 0.1188
Epoch 749/800
1/1 [==============================] - 0s 100ms/step - loss: 0.0720 - val_loss: 0.1658
Epoch 750/800
1/1 [==============================] - 0s 110ms/step - loss: 0.0752 - val_loss: 0.1381
Epoch 751/800
1/1 [==============================] - 0s 80ms/step - loss: 0.0679 - val_loss: 0.1321
Epoch 752/800
1/1 [==============================] - 0s 160ms/step - loss: 0.0641 - val_loss: 0.1689
Epoch 753/800
1/1 [==============================] - 0s 170ms/step - loss: 0.0687 - val_loss: 0.1387
Epoch 754/800
1/1 [==============================] - 0s 152ms/step - loss: 0.0742 - val_loss: 0.1461
Epoch 755/800
1/1 [==============================] - 0s 156ms/step - loss: 0.0800 - val_loss: 0.1621
Epoch 756/800
1/1 [==============================] - 0s 101ms/step - loss: 0.0762 - val_loss: 0.1279
Epoch 7

In [9]:
for c in chars:
    try: os.mkdir('model/' + c)
    except: pass
    autoencoder[c].save('model/' + c + '/')
    pd.DataFrame(autoencoder[c].history.history).to_csv('model/' + c + '/history.csv')

# EVALUATE DS1

In [10]:
test_data = {}
dataset = {}
filenames = {}

test_path = '../test-set/'

for c in chars:
    files = os.listdir(test_path + c + '/')
    test_data_ = []
    for f in files:
        img = np.asarray(Image.open(test_path + c + '/' + f).convert('L')).reshape((64,64,1))
        test_data_.append(img)
    test_data_ = np.asarray(test_data_) / 255
    test_data[c] = test_data_
    filenames[c] = files

In [11]:
results = {}
for c in chars:
    results[c] = autoencoder[c].model.evaluate(test_data[c],test_data[c])

print(results)

2/2 [==============================] - 0s 18ms/step - loss: 0.1391
{'yo': 0.15251781046390533, 'yorozu': 0.13911336660385132}
